# Laboratory LCA of Hydrothermal Carbonization  
This Jupyter notebook is for conducting an explaratory life cycle accessment (LCA) analysis to identify areas of sensitivity and further determine the optimal conditions for upscaling hydrothermal carbonization (HTC) under deep uncertainty. This notebook is organized in three sections: 
1) Project Setup: Importing main packages & ecoinvent databases 
2) Baseline LCA Model: Developing a baseline LCA model for evaluating environmental impacts, in the form of carbon,  for basic scenarios. 
3) Uncertainty Analysis: Performing a Monte Carlo simulation as a method of exploring optimal conditions for HTC to consider upon upscaling & futrue decision making, in consideration of lab-scale data uncertainty, utilizing a deterministic model.  

This notebook serves as a supplementary analysis to ____

## 1: Project Setup
Installing required packages, setting up projects, and importing ecoinvent databases. Be sure to activate the virtual environment and install requirements.txt.  

In [1]:
# Installing main packages
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# Importing BW2 packages
import brightway2 as bw
from bw2data.parameters import ProjectParameter, ParameterManager, ActivityParameter

In [3]:
# Import BW25 packages. 
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as bwa

In [4]:
# Creating a new project: HTC-uncertainty 
bd.projects.set_current('HTC-uncertainty')

In [5]:
# Installing biosphere flows 
bi.bw2setup()

Biosphere database already present!!! No setup is needed


In [6]:
# Getting main folder with ecoinvent files. Replace this with your own directory.  
def get_ecoinvent_directory(): 
    return "../../../../../../../demol/Documents/Research/LCA/ecoinvent_v3.10"

In [7]:
def write_data(ei_path, ei_name): 
    if ei_name in bd.databases:
        print("Database has already been imported.")
    else:
        ei_importer = bi.SingleOutputEcospold2Importer(ei_path, ei_name, use_mp = False)
        # Should you have a problem with "unlinked exchanges"
        try: 
            ei_importer.add_unlinked_flows_to_biosphere_database() 
        except Exception as e:
            print(f"An error occurred: {e}")
        ei_importer.apply_strategies()
        ei_importer.statistics()
        ei_importer.write_database()
        print(str(ei_name) + " database is completed!")

In [8]:
def create_databases():
    for folder in os.listdir(get_ecoinvent_directory()):
        folder_path = os.path.join(get_ecoinvent_directory(), folder + "/datasets")
        # Only using cutoff for this analysis 
        if os.path.isdir(folder_path) and 'cutoff' in folder:
            write_data(folder_path, folder_path.split('\\')[-1])
    print(bd.databases)

In [15]:
create_databases()

Extracting ecospold2 files:
0% [##############################] 100% | ETA: 00:00:00 | Item ID: fffee1f5-12b7-5
Total time elapsed: 00:12:46


Title: Extracting ecospold2 files:
  Started: 06/17/2024 10:42:42
  Finished: 06/17/2024 10:55:28
  Total time elapsed: 00:12:46
  CPU %: 16.10
  Memory %: 22.19
Extracted 23523 datasets in 769.81 seconds


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 06/17/2024 10:55:42
  Finished: 06/17/2024 10:55:42
  Total time elapsed: 00:00:00
  CPU %: 85.20
  Memory %: 23.88
Applying strategy: link_iterable_by_fields
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strateg

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:03


Title: Writing activities to SQLite3 database:
  Started: 06/17/2024 10:56:09
  Finished: 06/17/2024 10:57:12
  Total time elapsed: 00:01:03
  CPU %: 39.40
  Memory %: 23.69
Created database: cutoff/datasets
cutoff/datasets database is completed!
Databases dictionary with 3 object(s):
	apos/datasets
	biosphere3
	cutoff/datasets


In [9]:
# Ensuring cutoff database has been imported correctly
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	cutoff/datasets

In [10]:
# Creating database reference to not override or intefere with cutoff model: 
eidb = bd.Database("cutoff/datasets")

In [11]:
# Creating database reference to not override or intefere with the biosphere model: 
biosphere3 = bw.Database('biosphere3')

In [81]:
# Checking for proper replication: 
print("The imported ecoinvent database is of type {} and has a length of {}.".format(type(eidb), len(eidb)))

The imported ecoinvent database is of type <class 'bw2data.backends.peewee.database.SQLiteBackend'> and has a length of 23541.


In [83]:
# Checking for proper replication of biosphere database 
print("The biosphere database is of type {} and has a length of {}.".format(type(biosphere3), len(biosphere3)))

The biosphere database is of type <class 'bw2data.backends.peewee.database.SQLiteBackend'> and has a length of 1060.


## 2: Baseline LCA
Conducting a baseline LCA, as a foreground system for determining environmental impacts for main feedstocks based on numerical averages. The feedstocks of interest are as follows:  
 - SRU Feedstock with water to reach an 85% moisture content
 - BSG Feedstock with water to reach at 85% moisture content
 - DCW Feedstock (already above 85% moisture content; no additional water needed)

### 2.1: Identifying Appropriate Flows
All hydrothermal carbonization models have the following inputs: 
1) Feedstock: It will be assumed that the feedstock is a waste product and has no corresponding environmental impacts. The process of transporting food waste is not considered for this analysis.  
2) Water: If needed, water is added to reach a moisture content. If not needed, water is 0. 
3) Electricity: Needed for stirring of an HTC reaction & for post-processing. 
4) Heat: Needed for the HTC reaction and post-processing steps. 
It will be assumed that the equipment itself has a long lifetime and any pre-processing for the equipment or environmental impacts for setting up or maintaining a system will not be explored in this analysis. 

Simultaneously, all hydrothermal carbonization models have the following outputs: 
1. Hydrochar (functional unit): The goal of this analysis is to determine the optimal method for producing a solid hydrochar (HC).  
2. Carbon Dioxde: Gas yields from HTC are assumed to be pure CO<sub>2</sub> are emitted to the atmosphere. 
3. Wastewater: Process Water from the HTC process that requires treatment at a wastewater treatment plant. 

In [13]:
# Searching for Electricity
for act in [act for act in eidb if 'market group for electricity, low voltage' in act['name']  and 'US' in act['location']]:
    print(act)

'market group for electricity, low voltage' (kilowatt hour, US, None)


In [14]:
# Assuming Heat uses the same source as electricity, based on Parr reactor specifications
# https://www.parrinst.com/products/stirred-reactors/series-4520-1-2l-bench-top-reactors/specifications/

for act in [act for act in eidb if 'market group for electricity, low voltage' in act['name']  and 'US' in act['location']]:
    print(act)

'market group for electricity, low voltage' (kilowatt hour, US, None)


In [15]:
# Searching for Water 
for act in [act for act in eidb if 'market for water, completely softened' in act['name']  and 'US' in act['location']]:
    print(act)

'market for water, completely softened' (kilogram, US, None)


In [16]:
# Searching for Wastewater 
for act in [act for act in eidb if 'treatment of wastewater, average, wastewater treatment' in act['name']  and 'RoW' in act['location']]:
    print(act)

'treatment of wastewater, average, wastewater treatment' (cubic meter, RoW, None)


In [77]:
# Searching for Carbon Dioxide Stream; in this case the second option is of interest.  
for flow in [flow for flow in biosphere3 if 'carbon dioxide, non-fossil' in flow['name'].lower()]:
    print(flow)

'Carbon dioxide, non-fossil' (kg, None, None)
'Carbon dioxide, non-fossil, resource correction' (kg, None, None)


### 2.2: Setting Up Product System
Creating product systems for main feedstocks

In [18]:
from feedstock import Feedstock, FeedstockManager, create_elementary_feedstocks
from model import get_feedstock_quantity, get_water_quantity, get_heat_needed, get_electricity_needed
from model import  get_co2_emissions, get_pw, get_parameter, get_post_processing

In [19]:
# If needed, delete existing parameters
for item in ProjectParameter.select(): 
    item.delete()

ProjectParameter.drop_table(safe=True, drop_sequences=True)
# create a new empty table of project parameters
ProjectParameter.create_table()

for name in ProjectParameter.select():
    name.print()

In [20]:
# If needed, delete existing activities
for act in [act for act in eidb if 'hydrochar production' in act['name']]:
    act.delete()
    
for act in [act for act in eidb if 'hydrochar production' in act['name']]:
    print(act)

In [21]:
# Re initializing elementary feedstocks if needed
elementary_feedstocks = create_elementary_feedstocks()

In [22]:
# Creating Activities for Different Feedstocks 
excluded_feedstocks = {"rawSRU", "rawBSG"}
for attr, feedstocks in elementary_feedstocks.__dict__.items():
    for feedstock in feedstocks:
        if feedstock.name not in excluded_feedstocks:
            # for temp in HTC_temp:
            #     for time in HTC_reaction_time:
            temp = feedstock.temp
            time = feedstock.time 
            hydrochar_production = eidb.new_activity(
                code=f'HTC_{feedstock.name}_{temp}C_{time}hr',
                name=f"hydrochar production, {feedstock.name}_{temp}C_{time}hr",
                location='US',
                unit="kg"
            )
            parameters = ParameterManager()
            yield_HC = get_parameter(f"hydrochar production, {feedstock.name}_{temp}C_{time}hr", 'HC_yield')
            gas_yield = get_parameter(f"hydrochar production, {feedstock.name}_{temp}C_{time}hr", 'gas_yield')
            project_data = [
                {
                    'name': 'feedstock_' + f'HTC_{feedstock.name}_{temp}C_{time}hr', 
                    'amount': get_feedstock_quantity(yield_HC, feedstock)
                }, 
                {
                    'name': 'water_' + f'HTC_{feedstock.name}_{temp}C_{time}hr', 
                    'amount': get_water_quantity(yield_HC, feedstock)
                }, 
                {
                    'name': 'heat_' + f'HTC_{feedstock.name}_{temp}C_{time}hr', 
                    'amount': get_heat_needed(feedstock, temp, time)
                },
                {
                    'name': 'electricity_' + f'HTC_{feedstock.name}_{temp}C_{time}hr', 
                    'amount': get_electricity_needed(feedstock, time, temp)
                }, 
                {
                    'name': 'co2_' + f'HTC_{feedstock.name}_{temp}C_{time}hr',
                    'amount': get_co2_emissions(yield_HC, gas_yield, feedstock)
                }, 
                {
                    'name': 'wastewater_' + f'HTC_{feedstock.name}_{temp}C_{time}hr', 
                    'amount': get_pw(yield_HC, gas_yield, feedstock, hc_placeholder=1)
                },
                {
                    'name': 'post_processing_' + f'HTC_{feedstock.name}_{temp}C_{time}hr', 
                    'amount': get_post_processing(hc_placeholder=1)
                }
            ]
            
            print(project_data)
            parameters.new_project_parameters(project_data)
            
            # for param in ProjectParameter.select():
            #     print(param, param.amount)
            
            # Feedstock 
            hydrochar_production.new_exchange(input=('biosphere3', 'feedstock'), amount=0, unit="kg", type='custom', formula= 'feedstock')
            hydrochar_production.save()
                                
            # Water 
            water = [act for act in eidb if 'market for water, completely softened' in act['name']  and 'US' in act['location']][0]
            hydrochar_production.new_exchange(input=water.key, amount=0, unit="kilogram", type='technosphere', 
                                                formula= 'water_' + f'HTC_{feedstock.name}_{temp}C_{time}hr').save()
            hydrochar_production.save()
            
            # Electricity
            electricity = [act for act in eidb if 'market group for electricity, low voltage' in act['name']  and 'US' in act['location']][0]
            hydrochar_production.new_exchange(input=electricity.key, amount=0, unit="kilowatt hour", type='technosphere', 
                                                formula='electricity_' + f'HTC_{feedstock.name}_{temp}C_{time}hr').save()
            hydrochar_production.save()
            
            # Heat 
            electricity = [act for act in eidb if 'market group for electricity, low voltage' in act['name']  and 'US' in act['location']][0]
            hydrochar_production.new_exchange(input=electricity.key, amount=0, unit="kilowatt hour", type='technosphere', 
                                                formula='heat_' + f'HTC_{feedstock.name}_{temp}C_{time}hr').save()
            hydrochar_production.save()
            
            # CO2 Emissions
            co2 = [flow for flow in biosphere3 if 'carbon dioxide, non-fossil' in flow['name'].lower()][1]
            hydrochar_production.new_exchange(input=co2.key, amount=0, unit="kilogram", type='biosphere', 
                                                formula='-1*co2_' + f'HTC_{feedstock.name}_{temp}C_{time}hr').save()
            hydrochar_production.save()
            
            # Wastewater 
            wastewater = [act for act in eidb if 'treatment of wastewater, average, wastewater treatment' in act['name']  and 'RoW' in act['location']][0]
            hydrochar_production.new_exchange(input=wastewater.key, amount=0, unit="cubic meter", type='technosphere', 
                                                formula='-1*wastewater_' + f'HTC_{feedstock.name}_{temp}C_{time}hr/1000').save()
            hydrochar_production.save()
            
            # Post-Processing
            electricity = [act for act in eidb if 'market group for electricity, low voltage' in act['name']  and 'US' in act['location']][0]
            hydrochar_production.new_exchange(input=electricity.key, amount=0, unit="kilowatt hour", type='technosphere', 
                                                formula='post_processing_' + f'HTC_{feedstock.name}_{temp}C_{time}hr').save()
            hydrochar_production.save()
            
            parameters.add_exchanges_to_group(f"hydrochar production, {feedstock.name}_{temp}C_{time}hr", hydrochar_production)
            ActivityParameter.recalculate_exchanges(f"hydrochar production, {feedstock.name}_{temp}C_{time}hr")
            print()        

[{'name': 'feedstock_HTC_stdSRU_190C_1hr', 'amount': 4.405149378057423}, {'name': 'water_HTC_stdSRU_190C_1hr', 'amount': 1.4013095047525619}, {'name': 'heat_HTC_stdSRU_190C_1hr', 'amount': 1.802243989661737}, {'name': 'electricity_HTC_stdSRU_190C_1hr', 'amount': 0.04770949807419546}, {'name': 'co2_HTC_stdSRU_190C_1hr', 'amount': 0.013643627895201066}, {'name': 'wastewater_HTC_stdSRU_190C_1hr', 'amount': 4.792815254914784}, {'name': 'post_processing_HTC_stdSRU_190C_1hr', 'amount': 28.815}]

[{'name': 'feedstock_HTC_stdSRU_190C_3hr', 'amount': 3.902283644770988}, {'name': 'water_HTC_stdSRU_190C_3hr', 'amount': 1.2413443205572885}, {'name': 'heat_HTC_stdSRU_190C_3hr', 'amount': 3.9700552622566088}, {'name': 'electricity_HTC_stdSRU_190C_3hr', 'amount': 0.1116802249166801}, {'name': 'co2_HTC_stdSRU_190C_3hr', 'amount': 0.019413962826294342}, {'name': 'wastewater_HTC_stdSRU_190C_3hr', 'amount': 4.124214002501982}, {'name': 'post_processing_HTC_stdSRU_190C_3hr', 'amount': 28.815}]

[{'name': 

In [24]:
# Cross Checking Exchanges 
for act in [act for act in eidb if 'hydrochar production' in act['name']]:
    print(act)
    for exc in act.exchanges():
        print(exc)
    print()

'hydrochar production, rawDCW_190C_3hr' (kg, US, None)
Exchange: 0.0 kilogram 'market for water, completely softened' (kilogram, US, None) to 'hydrochar production, rawDCW_190C_3hr' (kg, US, None)>
Exchange: 0.4512728255547088 kilowatt hour 'market group for electricity, low voltage' (kilowatt hour, US, None) to 'hydrochar production, rawDCW_190C_3hr' (kg, US, None)>
Exchange: 13.861886998120793 kilowatt hour 'market group for electricity, low voltage' (kilowatt hour, US, None) to 'hydrochar production, rawDCW_190C_3hr' (kg, US, None)>
Exchange: -0.14313988821754794 kg 'Carbon dioxide, non-fossil' (kg, None, None) to 'hydrochar production, rawDCW_190C_3hr' (kg, US, None)>
Exchange: -0.05331388615338748 cubic meter 'treatment of wastewater, average, wastewater treatment' (cubic meter, RoW, None) to 'hydrochar production, rawDCW_190C_3hr' (kg, US, None)>
Exchange: 28.815 kilowatt hour 'market group for electricity, low voltage' (kilowatt hour, US, None) to 'hydrochar production, rawDCW_1

### 2.3: Life Cycle Assessment

In [25]:
[m for m in bw.methods if 'TRACI' in str(m)]

[('TRACI', 'human health', 'non-carcinogenics'),
 ('TRACI', 'human health', 'respiratory effects, average'),
 ('TRACI', 'human health', 'carcinogenics'),
 ('TRACI', 'environmental impact', 'eutrophication'),
 ('TRACI', 'environmental impact', 'global warming'),
 ('TRACI', 'environmental impact', 'ecotoxicity'),
 ('TRACI', 'environmental impact', 'photochemical oxidation'),
 ('TRACI', 'environmental impact', 'acidification'),
 ('TRACI', 'environmental impact', 'ozone depletion')]

In [26]:
method_key = [m for m in bw.methods if 'TRACI' in str(m) and 'global warming' in str(m)][0]
activity = [act for act in eidb if 'hydrochar production' in act['name']][0]
functional_unit = { activity:1e-3}

In [27]:
lca = bw.LCA(functional_unit, method_key)

In [28]:
lca.lci()
lca.lcia()

In [29]:
lca.demand

{'hydrochar production, stdBSG_250C_3hr' (kg, US, None): 0.001}

In [30]:
lca.method

('TRACI', 'environmental impact', 'global warming')

In [31]:
print("The {} process accounts for {:f} {}.".format(
    list(functional_unit.keys())[0]['name'],
    lca.score,
    bw.methods.get(method_key).get('unit')
    ))

The hydrochar production, stdBSG_250C_3hr process accounts for 0.016480 kg CO2-Eq.


In [32]:
total_impact = lca.score

In [33]:
[act for act in eidb if 'hydrochar production' in act['name']][0]

'hydrochar production, rawDCW_190C_3hr' (kg, US, None)

In [34]:
exchange_contributions = []
for exc in activity.exchanges():
    # Create a temporary LCA object for each exchange
    temp_lca = bw.LCA({exc.input: exc['amount']}, method_key)
    print(exc.input)
    if 'Carbon dioxide' in str(exc.input): 
        # pass
        temp_lca.lci()
        temp_lca.lcia()
    else: 
        temp_lca.lci()
        temp_lca.lcia()
        
        # Calculate the contribution of this exchange
        contribution = temp_lca.score
        
        # Store the result
        exchange_contributions.append({
            'exchange': exc,
            'contribution': contribution,
            'percentage': (contribution * 0.001 / total_impact) * 100
        })

# Print the results
for result in exchange_contributions:
    print(f"Exchange: {result['exchange']}")
    print(f"Contribution: {result['contribution']:.4f}")
    print(f"Percentage of total impact: {result['percentage']:.2f}%")
    print('-' * 40)

'market for water, completely softened' (kilogram, US, None)
'market group for electricity, low voltage' (kilowatt hour, US, None)
'market group for electricity, low voltage' (kilowatt hour, US, None)
'Carbon dioxide, non-fossil, resource correction' (kg, None, None)


AllArraysEmpty: 

In [68]:
# Print the results to see if the flow you are looking for is present
for flow in [flow for flow in biosphere if 'carbon dioxide, non-fossil' in flow['name'].lower()]:
    print(flow.biosphere.__dict__)
    
    # print(flow.lca())
    # test_lca = bw.LCA({flow:1}, method_key)
    # test_lca.lci()
    # test_lca.lcia()
    
    # print(flow['name'], '-', flow['categories'])
    

{}
{}
